<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/IT_EFAS_INDU_SPECIAL_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date

import xml.etree.ElementTree as ET
from urllib.request import urlopen

!pip install yfinance
import yfinance as yf

!pip install finance-datareader
import FinanceDataReader as fdr

##### 조회년월
queryYYYYMM = '202211'

##### 조회분기
queryYYYYQQ = '2022Q3'

##### ECOS KEY
key = 'MTSA07N58C5X8EB4LMJL'

##### KOSIS KEY
keyKOSIS = '7AA431187D1220S0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 537 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
## 자동차 산업 (출처 : 공공데이터포털 - 산업통상자원부_전체 자동차 산업 현황)
# MOTOR_PROD_QUAN : 자동차산업동향-생산수량(대)
# MOTOR_DOME_QUAN : 자동차산업동향-내수판매량(대)
# MOTOR_EXPORT_QUAN : 자동차산업동향-수출수량(대)
# MOTOR_EXPORT_SALES : 자동차산업동향-수출금액(천 달러)
# https://api.odcloud.kr/api/15051118/v1/uddi:7b4196ec-6862-4193-a7c8-c4f217150db8_202003251527?page=1&perPage=1000&serviceKey=6No4JLBZPafGmvs0YQ%2FWiHSNqVrvqqAox0frrHQJbt%2Fvc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q%3D%3D

url = f'https://api.odcloud.kr/api/15051118/v1/uddi:7b4196ec-6862-4193-a7c8-c4f217150db8_202003251527?page=1&perPage=1000&serviceKey=6No4JLBZPafGmvs0YQ%2FWiHSNqVrvqqAox0frrHQJbt%2Fvc6QT4xzf2Idnc9CaktyAdlI3FYGILmdhVIH9u5ck2Q%3D%3D'
r = requests.get(url)
jo = r.json()
motorDataFrame = pd.DataFrame(jo['data'])

motorDataFrame = motorDataFrame[['기간', '생산수량', '내수판매수량', '수출수량', '수출금액(천달러)']] ## 원하는 컬럼만 뽑아냄
motorDataFrame.rename(columns = {'기간':'PERIOD', '생산수량':'MOTOR_PROD_QUAN', '내수판매수량':'MOTOR_DOME_QUAN', '수출수량':'MOTOR_EXPORT_QUAN', '수출금액(천달러)':'MOTOR_EXPORT_SALES'}, inplace = True)
motorDataFrame['MOTOR_PROD_QUAN'] = pd.to_numeric(motorDataFrame['MOTOR_PROD_QUAN'])
motorDataFrame['MOTOR_DOME_QUAN'] = pd.to_numeric(motorDataFrame['MOTOR_DOME_QUAN'])
motorDataFrame['MOTOR_EXPORT_QUAN'] = pd.to_numeric(motorDataFrame['MOTOR_EXPORT_QUAN'])
motorDataFrame['MOTOR_EXPORT_SALES'] = pd.to_numeric(motorDataFrame['MOTOR_EXPORT_SALES']).round(0)
motorDataFrame = motorDataFrame.set_index('PERIOD')
motorDataFrame = motorDataFrame.loc['2010-01':] ## 2010년 1월 이후 데이터를 뽑아냄
motorDataFrame['STD_YM'] = motorDataFrame.index.astype(str).str.slice(0,4) + motorDataFrame.index.astype(str).str.slice(5,7)
motorDataFrame = motorDataFrame.set_index('STD_YM')

motorDataFrame

,MOTOR_PROD_QUAN,MOTOR_DOME_QUAN,MOTOR_EXPORT_QUAN,MOTOR_EXPORT_SALES
STD_YM,,,,
201501,376018,112407,248613,3630909.0
201502,317746,104053,209147,3040431.0
201503,414313,128103,275999,3996144.0
201504,424401,134631,282044,4047264.0
201505,365643,122152,247343,3485506.0
...,...,...,...,...
202108,234963,106796,130213,2703340.0
202109,229423,92343,151214,3209295.0
202110,263723,107101,160123,3463588.0


In [ ]:
### 조선 산업 (출처 : e나라지표-수주량, 건조량, 수주잔량   /   클락슨-신조선가지수, 선복량, 해체량)
# SHIP_ORDER_AMOUNT : 선박 수주량(CGT)
# SHIP_TONNAGE : 선박 건조량(CGT)
# SHIP_KAJI : (클락슨)신조선가지수
# SHIP_BACKLOG : 선박 수주잔량(CGT)
# SHIP_BOTTOMS : (클락슨)선박 선복량(DWT)
# SHIP_BREAKUP : (클락슨)선박 해체량(DWT)
# (Ex) http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId=7AA431187D1220S0&statsCode=115102&period=201001:202211'

key = '7AA431187D1220S0'
url = f'http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId={key}&statsCode=115102&period=201001:{queryYYYYMM}'
response = urlopen(url).read()
xtree = ET.fromstring(response)

## '표' tag에서 '주기'값이 '월'인 tree를 찾아서 월별 데이터 시작 지점을 찾음
for neighbor in xtree.iter("표"):
  if neighbor.attrib['주기'] == '월':
    rootMonth = neighbor

## '항목' tag에서 '이름'값에 따라 항목(수주랑, 건조량, 수주잔량)별 데이터 시작 지점을 찾음
for child in rootMonth.iter("항목"):
  if child.attrib['이름'] == '수주량':
    sooju = child
  elif child.attrib['이름'] == '건조량':
    gunjo = child
  elif child.attrib['이름'] == '수주잔량':
    soojujan = child

## 수주량 데이터 적제
period = [x.attrib["주기"] for x in sooju]
soojuValue = [x.text for x in sooju]

## 건조량 데이터 적제
gunjoValue = [x.text for x in gunjo]

## 수주잔량 데이터 적제
soojujanValue = [x.text for x in soojujan]

shipDataFrame = pd.DataFrame()
shipDataFrame['STD_YM'] = period
shipDataFrame['SHIP_ORDER_AMOUNT'] = soojuValue  # 수주량
shipDataFrame['SHIP_TONNAGE'] = gunjoValue  # 건조량
shipDataFrame['SHIP_BACKLOG'] = soojujanValue # 수주잔량

## 클락슨 데이터 들어갈 자리 삽입
shipDataFrame.insert(3, 'SHIP_KAJI', '')
shipDataFrame.insert(5, 'SHIP_BOTTOMS', '')
shipDataFrame["SHIP_BREAKUP"]=""

shipDataFrame = shipDataFrame.set_index('STD_YM')
shipDataFrame

,SHIP_ORDER_AMOUNT,SHIP_TONNAGE,SHIP_KAJI,SHIP_BACKLOG,SHIP_BOTTOMS,SHIP_BREAKUP
STD_YM,,,,,,
201501,79,147,,3475,,
201502,100,101,,3374,,
201503,116,81,,3333,,
201504,61,126,,3362,,
201505,120,91,,3296,,
...,...,...,...,...,...,...
202109,109.9,72.0,,2864,,
202110,112.8,96.9,,2879,,
202111,110.1,61.2,,2929,,


In [ ]:
## 철강 산업 (출처 : KOSIS철강통계조사-조강생산량(천 톤)   /   철강협회-수출량, 내수판매량   /   Fred(IMF)-철광석가격)
# STEEL_EXPORT_QUAN : (철강협회)철강 수출량(톤)
# STEEL_DOME_QUAN : (철강협회)철강 내수판매량(톤)
# STEEL_CRUDE_QUAN : 조강생산량(천 톤)
# STEEL_PRICE : (Fred)철광석 가격(Million dollar per Metric Ton)

url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=16363AAA0+&objL1=15363AA1AA+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201001&endPrdDe={queryYYYYMM}&orgId=363&tblId=TX_36301_A000'
r = requests.get(url)
jo = r.json()
ironDataFrame = pd.DataFrame(jo)
# 원하는 컬럼만 뽑아냄(PRD_DE : 기준년월, DT : 조강생산량(천 톤))
ironDataFrame = ironDataFrame[['PRD_DE', 'DT']] 
ironDataFrame['DT'] = pd.to_numeric(ironDataFrame['DT'])
ironDataFrame.rename(columns = {'PRD_DE':'STD_YM'}, inplace = True)
ironDataFrame.rename(columns = {'DT':'STEEL_CRUDE_QUAN'}, inplace = True)
ironDataFrame = ironDataFrame.set_index('STD_YM')

# 철강협회 데이터 들어갈 자리 삽입
ironDataFrame.insert(0, 'STEEL_EXPORT_QUAN', '')
ironDataFrame.insert(1, 'STEEL_DOME_QUAN', '')

## 철광석 가격(Iron ore) : fred(https://fred.stlouisfed.org/)에서 PIORECRUSDM 데이터 가져옴
dfSteelPrice = fdr.DataReader('FRED:PIORECRUSDM', '2010-01')
dfSteelPrice['STD_YM'] = dfSteelPrice.index.astype(str).str.slice(0,4) + dfSteelPrice.index.astype(str).str.slice(5,7)
dfSteelPrice = dfSteelPrice.set_index('STD_YM')

ironDataFrame = pd.merge(left = ironDataFrame, right = dfSteelPrice, left_index = True, right_index = True, how = "outer")
ironDataFrame


,STEEL_EXPORT_QUAN,STEEL_DOME_QUAN,STEEL_CRUDE_QUAN,PIORECRUSDM
STD_YM,,,,
201001,,,4570.0,125.914286
201002,,,4024.0,127.615000
201003,,,5087.0,139.769565
201004,,,5126.0,172.468182
201005,,,5246.0,161.347619
...,...,...,...,...
202206,,,5585.0,131.045455
202207,,,6276.0,109.047619
202208,,,5934.0,109.065217


In [ ]:
## 석유화학 산업 (출처 : 한국석유화학협회-3대유도품수출량, 3대유도품내수판매량)
# PETRO_EXPORT_QUAN : (석유화학협회)3대유도품수출량
# PETRO_DOME_QUAN : (석유화학협회)3대유도품내수판매량
petroDataFrame = pd.DataFrame()
petroDataFrame['STD_YM'] = ironDataFrame.index
petroDataFrame['PETRO_EXPORT_QUAN'] = ""
petroDataFrame['PETRO_DOME_QUAN'] = ""
petroDataFrame = petroDataFrame.set_index('STD_YM')

petroDataFrame

,PETRO_EXPORT_QUAN,PETRO_DOME_QUAN
STD_YM,,
201001,,
201002,,
201003,,
201004,,
201005,,
...,...,...
202206,,
202207,,
202208,,


In [ ]:
## 반도체 산업 (출처 : 한국반도체산업협회-수입현황,수출현황   /   야후파이낸스-필라델피아반도체지수)
# SEMI_IMPORT : 반도체 수출입동향-수입
# SEMI_EXPORT : 반도체 수출입동향-수출
# SEMI_GLOBAL_SALES : 반도체 글로벌 매출전망
# SEMI_64GB_MLC_SPOT : 64Gb MLC 현물가
# SEMI_64GB_MLC_FIX : 64Gb MLC 고정가
# SEMI_128GB_MLC_SPOT : 128Gb MLC 현물가
# SEMI_128GB_MLC_FIX : 128Gb MLC 고정가
# SEMI_DRAM_DDR4_8GB_SPOT : PC DRAM DDR4 8Gb(1Gx8) 현물가
# SEMI_DRAM_DDR4_8GB_FIX : PC DRAM DDR4 8Gb(1Gx8) 고정가
# SEMI_PHILADELPHIA : 필라델피아지수

# 필라델피아지수
semiDataFrame = yf.download('^SOX',start = '2010-01-01')
semiDataFrame = semiDataFrame[['Adj Close']]
semiDataFrame['STD_YM'] = semiDataFrame.index.astype(str).str.slice(0,4) + semiDataFrame.index.astype(str).str.slice(5,7)
semiDataFrame = semiDataFrame.groupby("STD_YM")['Adj Close'].mean() # 월평균
semiDataFrame = semiDataFrame.to_frame()
semiDataFrame.rename(columns = {'Adj Close':'SEMI_PHILADELPHIA'}, inplace = True)

# 기타 데이터 삽입
semiDataFrame.insert(0, 'SEMI_IMPORT', '')
semiDataFrame.insert(1, 'SEMI_EXPORT', '')
semiDataFrame.insert(2, 'SEMI_GLOBAL_SALES', '')
semiDataFrame.insert(3, 'SEMI_64GB_MLC_SPOT', '')
semiDataFrame.insert(4, 'SEMI_64GB_MLC_FIX', '')
semiDataFrame.insert(5, 'SEMI_128GB_MLC_SPOT', '')
semiDataFrame.insert(6, 'SEMI_128GB_MLC_FIX', '')
semiDataFrame.insert(7, 'SEMI_DRAM_DDR4_8GB_SPOT', '')
semiDataFrame.insert(8, 'SEMI_DRAM_DDR4_8GB_FIX', '')

semiDataFrame

[*********************100%***********************]  1 of 1 completed


<ipython-input-31-e8e407d5ac34>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semiDataFrame['STD_YM'] = semiDataFrame.index.astype(str).str.slice(0,4) + semiDataFrame.index.astype(str).str.slice(5,7)


,SEMI_IMPORT,SEMI_EXPORT,SEMI_GLOBAL_SALES,SEMI_64GB_MLC_SPOT,SEMI_64GB_MLC_FIX,SEMI_128GB_MLC_SPOT,SEMI_128GB_MLC_FIX,SEMI_DRAM_DDR4_8GB_SPOT,SEMI_DRAM_DDR4_8GB_FIX,SEMI_PHILADELPHIA
STD_YM,,,,,,,,,,
201001,,,,,,,,,,349.100549
201002,,,,,,,,,,331.668990
201003,,,,,,,,,,358.039368
201004,,,,,,,,,,386.359719
201005,,,,,,,,,,356.888315
...,...,...,...,...,...,...,...,...,...,...
202208,,,,,,,,,,2939.390434
202209,,,,,,,,,,2529.717134
202210,,,,,,,,,,2331.055734


In [ ]:
## 디스플레이 산업 (출처 : 한국디스플레이산업협회-수입현황,수출현황,55인치LCDTV패널평균가격,OLED핸드폰평균가격,LCD평균가격,AMOLED평균가격,MicroLED평균가격,디스플레이수출동향(LCD비중),디스플레이수출동향(OLED비중))
# DISP_IMPORT : 디스플레이 수출입동향-수입
# DISP_EXPORT : 디스플레이 수출입동향-수출
# DISP_55INCH_PRICE : 55인치 LCDTV패널 평균가격
# DISP_OLED_CEL_PRICE : OLED 핸드폰 평균가격
# DISP_LED_PRICE : LCD 평균가격
# DISP_AMOLED_RRICE : AMOLED 평균가격
# DISP_MICROLED_PRICE : MicroLED 평균가격,
# DISP_LCD_EXPORT_RATIO : 디스플레이 수출동향(LCD비중),
# DISP_OLED_EXPORT_RATIO : 디스플레이 수출동향(OLED비중)

dispDataFrame = pd.DataFrame()
dispDataFrame['STD_YM'] = semiDataFrame.index
dispDataFrame['DISP_IMPORT'] = ""
dispDataFrame['DISP_EXPORT'] = ""
dispDataFrame['DISP_55INCH_PRICE'] = ""
dispDataFrame['DISP_OLED_CEL_PRICE'] = ""
dispDataFrame['DISP_LED_PRICE'] = ""
dispDataFrame['DISP_AMOLED_RRICE'] = ""
dispDataFrame['DISP_MICROLED_PRICE'] = ""
dispDataFrame['DISP_LCD_EXPORT_RATIO'] = ""
dispDataFrame['DISP_OLED_EXPORT_RATIO'] = ""
dispDataFrame = dispDataFrame.set_index('STD_YM')

dispDataFrame

,DISP_IMPORT,DISP_EXPORT,DISP_55INCH_PRICE,DISP_OLED_CEL_PRICE,DISP_LED_PRICE,DISP_AMOLED_RRICE,DISP_MICROLED_PRICE,DISP_LCD_EXPORT_RATIO,DISP_OLED_EXPORT_RATIO
STD_YM,,,,,,,,,
201001,,,,,,,,,
201002,,,,,,,,,
201003,,,,,,,,,
201004,,,,,,,,,
201005,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
202208,,,,,,,,,
202209,,,,,,,,,
202210,,,,,,,,,


In [ ]:
## 정유 산업 (출처 : ECOS-수입현황,수출현황,내수판매량   /   싱가포르정제마진,두바이크랙마진)
# OIL_IMPORT_QUAN : 정유 수출입동향-수입
# OIL_EXPORT_QUAN  : 정유 수출입동향-수출
# OIL_DOME_QUAN : 정유 수출입동향-내수판매량
# OIL_SINGAPORE_MARGIN : 싱가포르 정제마진
# OIL_CRACK_MARGIN : 두바이 크랙마진

key = 'MTSA07N58C5X8EB4LMJL'

def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'])
  df = df.set_index('STD_YM')
  return df

# (1) 정유 수출입동향-수입
# 8.4.11. 석유제품수급 : [901Y073][M]   /   수입[I46B][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202211/I46B
dfOilImportQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46B")
dfOilImportQuan.rename(columns = {'DATA_VALUE':'OIL_IMPORT_QUAN'}, inplace = True)

# (2) 정유 수출입동향-수출
# 8.4.11. 석유제품수급 : [901Y073][M]   /   수출[I46D][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202209/I46B
dfOilExportQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46D")
dfOilExportQuan.rename(columns = {'DATA_VALUE':'OIL_EXPORT_QUAN'}, inplace = True)

# (3) 정유 수출입동향-내수판매
# 8.4.11. 석유제품수급 : [901Y073][M]   /   국내소비[I46C][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202209/I46B
dfOilDomeQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46C")
dfOilDomeQuan.rename(columns = {'DATA_VALUE':'OIL_DOME_QUAN'}, inplace = True)

# compile the list of dataframes you want to merge
df_list = [dfOilImportQuan, dfOilExportQuan, dfOilDomeQuan]
# grab first dataframe
oilDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    oilDataFrame = pd.merge(left = oilDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")

## 기타 데이터 삽입
oilDataFrame['OIL_SINGAPORE_MARGIN'] = ""
oilDataFrame['OIL_CRACK_MARGIN'] = ""

oilDataFrame

,OIL_IMPORT_QUAN,OIL_EXPORT_QUAN,OIL_DOME_QUAN,OIL_SINGAPORE_MARGIN,OIL_CRACK_MARGIN
STD_YM,,,,,
201001,100370,23532,69318,,
201002,89701,22709,62758,,
201003,90927,25249,66693,,
201004,94342,28540,65588,,
201005,99923,27471,65202,,
...,...,...,...,...,...
202206,100599,35849,69964,,
202207,132306,47132,82160,,
202208,125323,50948,83666,,


In [4]:
## 해운 산업 (클락슨-BDI,SCFI,VLCC,CCFI   /   통계청KOSIS-컨테이너수송현황,수출입화물수송)
# HAEWOON_BDI : 해운 운임지수-BDI(벌크선)
# HAEWOON_SCFI : 해운 운임지수-SCFI(컨테이너선)
# HAEWOON_VLCC : 해운 운임지수-VLCC-WS(유조선)
# HAEWOON_CCFI : 해운 운임지수-CCFi
# HAEWOON_CONT : 해운 운임지수-컨테이너수송현황 (TEU)
# HAEWOON_TRANS : 해운 운임지수-수출입 화물수송 (R/T)
# HAEWOON_DUBAI : 두바이 유가
# HAEWOON_DOME_TRANS : 해운 국내 화물수송량 (톤)


## (통계청KOSIS) 컨테이너 화물수송 현황
startYYYYMM = 201101
endYYYYMM = int(queryYYYYMM)
yyyyMM_list = list(range(startYYYYMM, endYYYYMM, 100))

def kosisContainerApi(yyyyMM):
  s_yyyyMM = str(yyyyMM)
  e_yyyyMM = str(yyyyMM + 11)
  url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114674T.0003+&objL1=13102114674A.01+13102114674A.02+13102114674A.03+13102114674A.04+13102114674A.05+13102114674A.06+13102114674A.07+13102114674A.08+13102114674A.09+13102114674A.10+13102114674A.11+13102114674A.12+13102114674A.13+13102114674A.14+13102114674A.15+13102114674A.16+13102114674A.17+13102114674A.18+13102114674A.19+13102114674A.20+13102114674A.21+13102114674A.22+13102114674A.23+13102114674A.24+13102114674A.25+13102114674A.26+13102114674A.27+13102114674A.28+13102114674A.29+13102114674A.30+&objL2=13102114674B.01+13102114674B.02+&objL3=13102114674C.01+13102114674C.02+13102114674C.03+13102114674C.04+13102114674C.05+&objL4=13102114674D.0006+&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={s_yyyyMM}&endPrdDe={e_yyyyMM}&orgId=146&tblId=DT_MLTM_1312'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo)
  df = df[['PRD_DE', 'C1_NM', 'C2_NM', 'C3_NM', 'DT']]  # C1_NM:항구명, C2_NM:국적구분(국적선/외국선), C3_NM:구분(내항/외항입항/외항출항/입항환적/출항환적, DT:화물수송량(R/T)
  df['DT'] = df['DT'].astype(float) # df['DT'] = pd.to_numeric(df['DT'])
  df.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_CONT'}, inplace = True)
  df = df.set_index('STD_YM')
  return df

dfCONT = pd.DataFrame()
for elem in yyyyMM_list:
  if(dfCONT.empty):
    dfCONT = kosisContainerApi(elem)
  else:
    dfCONT = pd.concat([dfCONT, kosisContainerApi(elem)])
dfCONT = dfCONT.groupby('STD_YM').sum()


## (통계청KOSIS) 수출입 화물수송 현황
# (수출입 따로 보는 경우) url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114675T.0001+&objL1=13102114675A.01+&objL2=13102114675B.01+&objL3=13102114675C.0002+13102114675C.0003+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201101&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1316'
# (수출입 합쳐 보는 경우) 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114675T.0001+&objL1=13102114675A.01+&objL2=13102114675B.01+&objL3=13102114675C.0001+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201101&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1316'
r = requests.get(url)
jo = r.json()
dfTRANS = pd.DataFrame(jo)
dfTRANS = dfTRANS[['PRD_DE', 'C3_NM', 'DT']]  # (수출입 따로 보는경우) C3_NM:수출/수입/전체,   (수출입 같이 보는경우)C_NM:전체
dfTRANS['DT'] = round(pd.to_numeric(dfTRANS['DT']) / 1000, 0) # 천톤단위로 변환
dfTRANS = dfTRANS[['PRD_DE', 'DT']]
dfTRANS.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_TRANS'}, inplace = True)
dfTRANS = dfTRANS.set_index('STD_YM')
# (수출입 따로 보는 경우)
#dfImport = pd.DataFrame()
#dfExport = pd.DataFrame()
#dfImport = df.query('C3_NM == "수입"')
#dfExport = df.query('C3_NM == "수출"')


## 유가(Dubai) 추가 (ECOS)
# 9.1.6.3. 국제상품가격 : [902Y003][M]   /   Dubai(현물) : [4010102][U$/bbl]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/902Y003/M/201001/202208/4010102
def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df = df.set_index('STD_YM')
  return df

dfDUBAI = ecosApiCall(f"902Y003/M/201001/{queryYYYYMM}/4010102")
dfDUBAI.rename(columns = {'DATA_VALUE':'HAEWOON_DUBAI'}, inplace = True)



## (통계청KOSIS) 국내화물수송량 : 화물수송실적 - 내항화물입항현황 - 연안화물선(입항) - 화물(단위: R/T)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114676T.0003+&objL1=13102114676A.01+&objL2=13102114676B.0002+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201006&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1314'
r = requests.get(url)
jo = r.json()
dfDomeTrans = pd.DataFrame(jo)
dfDomeTrans = dfDomeTrans[['PRD_DE', 'ITM_NM', 'C1_NM', 'C2_NM', 'DT']]  # ITM_NM:화물, C1_NM:총계, C2_NM:연안화물선(입항)
dfDomeTrans['DT'] = round(pd.to_numeric(dfDomeTrans['DT']) / 1000, 0) # 천톤단위로 변환
dfDomeTrans = dfDomeTrans[['PRD_DE', 'DT']]
dfDomeTrans.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_DOME_TRANS'}, inplace = True)
dfDomeTrans = dfDomeTrans.set_index('STD_YM')


# compile the list of dataframes you want to merge
df_list = [dfCONT, dfTRANS, dfDUBAI, dfDomeTrans]
# grab first dataframe
haewoonDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    haewoonDataFrame = pd.merge(left = haewoonDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")


# 기타 데이터 삽입
haewoonDataFrame.insert(0, 'HAEWOON_BDI', '')
haewoonDataFrame.insert(1, 'HAEWOON_SCFI', '')
haewoonDataFrame.insert(2, 'HAEWOON_VLCC', '')
haewoonDataFrame.insert(3, 'HAEWOON_CCFI', '')

haewoonDataFrame

,HAEWOON_BDI,HAEWOON_SCFI,HAEWOON_VLCC,HAEWOON_CCFI,HAEWOON_CONT,HAEWOON_TRANS,HAEWOON_DUBAI,HAEWOON_DOME_TRANS
STD_YM,,,,,,,,
201001,,,,,NaN,NaN,71.84,NaN
201002,,,,,NaN,NaN,74.33,NaN
201003,,,,,NaN,NaN,79.05,NaN
201004,,,,,NaN,NaN,85.65,NaN
201005,,,,,NaN,NaN,73.28,NaN
...,...,...,...,...,...,...,...,...
202207,,,,,2545459.50,114097.0,105.8,7252.0
202208,,,,,2410835.25,111162.0,95.55,7159.0
202209,,,,,2046657.50,97038.0,90.1,6600.0


In [ ]:
## 건설 산업 (ECOS-건설국내수주량, 건설국내기성액, 건설해외수주량, 시도별건축착공현황, 미분양주택현황(서울,경기), 미분양주택현황(지방), 전국종합주택매매가격지수)
# CONST_DOME_ORDER_QUAN : 건설국내수주량
# CONST_REAL_PRICE : 건설국내기성액
# CONST_OVERSEA_ORDER_QUAN : 건설해외수주량
# CONST_BREAK_STATE : 시도별건축착공현황
# CONST_UNSOLD_URBAN : 미분양주택현황(서울,경기)
# CONST_UNSOLD_RURAL : 미분양주택현황(지방)
# CONST_HOUSE_PRICE : 전국종합주택매매가격지수

def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'])
  df = df.set_index('STD_YM')
  return df

## 건설국내수주액
# 8.4.1. 국내건설수주액 : [901Y020][M]   /   총수주액[I42A][백만 원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y020/M/201001/202209/I42A
dfConstDomeOrderQuan = ecosApiCall(f"901Y020/M/201001/{queryYYYYMM}/I42A")
dfConstDomeOrderQuan.rename(columns = {'DATA_VALUE':'CONST_DOME_ORDER_QUAN'}, inplace = True)

## 국내건설기성액
# 8.4.2. 건설기성액 : [901Y104][M]   /   총기성액[I48A][백만 원]    /   경상[I37A]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y104/M/201001/202209/I48A/I37A
dfConstRealPrice = ecosApiCall(f"901Y104/M/201001/{queryYYYYMM}/I48A/I37A")
dfConstRealPrice.rename(columns = {'DATA_VALUE':'CONST_REAL_PRICE'}, inplace = True)

## 시도별건축착공현황
# 8.4.3 건축착공현황 : [901Y103][M]   /   동수[2][동수]
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y103/M/201001/202209/2
dfConstBreakState = ecosApiCall(f"901Y103/M/201001/{queryYYYYMM}/2/I47AA")
dfConstBreakState.rename(columns = {'DATA_VALUE':'CONST_BREAK_STATE'}, inplace = True)

## 미분양주택현황(서울,경기) 
# 8.4.5 미분양주택현황 : [901Y074][M]   /   서울[I410B][호], 경기[I410I][호]
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410B
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410I
dfConstUnsoldUrban = ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410B") + ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410I")
dfConstUnsoldUrban.rename(columns = {'DATA_VALUE':'CONST_UNSOLD_URBAN'}, inplace = True)
 
## 미분양주택현황(지방) 
# 8.4.5 미분양주택현황 : [901Y074][M]   /   전국[I410A][호]
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410A
dfConstUnsoldRural = ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410A")
dfConstUnsoldRural.rename(columns = {'DATA_VALUE':'CONST_UNSOLD_RURAL'}, inplace = True)
dfConstUnsoldRural = dfConstUnsoldRural - dfConstUnsoldUrban


## 건설해외수주량(출처: KOSIS)
url = f'http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId={keyKOSIS}&statsCode=122103&period=201001:{queryYYYYMM}'
response = urlopen(url).read()
xtree = ET.fromstring(response)

# '표' tag에서 '주기'값이 '월'인 tree를 찾아서 월별 데이터 시작 지점을 찾음
for neighbor in xtree.iter("표"):
  if neighbor.attrib['주기'] == '월':
    for elem in neighbor.iter("분류1"):
       if elem.attrib['이름'] == '합계':
         rootMonth = elem

# 해외건설수주액 데이터 적제
period = [x.attrib["주기"] for x in rootMonth]
value = [x.text for x in rootMonth]

dfConstOversea = pd.DataFrame()
dfConstOversea['STD_YM'] = period
dfConstOversea['CONST_OVERSEA_ORDER_QUAN'] = value
dfConstOversea['STD_YM'] = dfConstOversea['STD_YM'].astype(int)
dfConstOversea = dfConstOversea.query("STD_YM >= 201001")
dfConstOversea['STD_YM'] = dfConstOversea['STD_YM'].astype(str)
dfConstOversea = dfConstOversea.set_index('STD_YM')


## 전국종합주택매매가격지수(출처: KOSIS 유형별 매매가격지수)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=sales+&objL1=00+&objL2=a0+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201001&endPrdDe={queryYYYYMM}&orgId=408&tblId=DT_40803_N0001'
r = requests.get(url)
jo = r.json()
dfConstHousePrice = pd.DataFrame(jo)
dfConstHousePrice = dfConstHousePrice[['PRD_DE', 'C2_NM', 'DT']]  # 원하는 컬럼만 뽑아냄(PRD_DE : 기준년월, DT : DATA_VALUE(100 = 202106))
dfConstHousePrice['DT'] = pd.to_numeric(dfConstHousePrice['DT'])
dfConstHousePrice.rename(columns = {'PRD_DE':'STD_YM', 'DT':'CONST_HOUSE_PRICE'}, inplace = True)
dfConstHousePrice = dfConstHousePrice[['STD_YM', 'CONST_HOUSE_PRICE']]
dfConstHousePrice = dfConstHousePrice.set_index('STD_YM')

# compile the list of dataframes you want to merge
df_list = [dfConstDomeOrderQuan, dfConstRealPrice, dfConstOversea, dfConstBreakState, dfConstUnsoldUrban, dfConstUnsoldRural, dfConstHousePrice]
# grab first dataframe
constDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    constDataFrame = pd.merge(left = constDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")

constDataFrame

,CONST_DOME_ORDER_QUAN,CONST_REAL_PRICE,CONST_OVERSEA_ORDER_QUAN,CONST_BREAK_STATE,CONST_UNSOLD_URBAN_x,CONST_UNSOLD_RURAL,CONST_UNSOLD_URBAN_y,CONST_HOUSE_PRICE
STD_YM,,,,,,,,
201001,7230174,6500576,22076325,NaN,22235.0,NaN,NaN,78.369310
201002,4863842,6149956,3414456,NaN,24280.0,NaN,NaN,78.573339
201003,6356428,7694396,2585419,NaN,23066.0,NaN,NaN,78.746871
201004,6664740,7345162,4057530,NaN,22961.0,NaN,NaN,78.884420
201005,8844915,7739757,847244,NaN,24306.0,NaN,NaN,78.903691
...,...,...,...,...,...,...,...,...
202206,24987168,14857547,NaN,NaN,4038.0,NaN,NaN,104.837210
202207,20836561,12226298,NaN,12553.0,3985.0,NaN,NaN,104.758261
202208,17506045,12619891,NaN,12046.0,3790.0,NaN,NaN,104.459050


In [ ]:
## 모두 합치기
df_list = [motorDataFrame, shipDataFrame, ironDataFrame, petroDataFrame, semiDataFrame, dispDataFrame, oilDataFrame, haewoonDataFrame, constDataFrame]
# grab first dataframe
IT_EFAS_INDU_SPECIAL_DB = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    IT_EFAS_INDU_SPECIAL_DB = pd.merge(left = IT_EFAS_INDU_SPECIAL_DB, right = to_merge, left_index = True, right_index = True, how = "outer")

today = date.today()
IT_EFAS_INDU_SPECIAL_DB.to_excel(f'{today}_IT_EFAS_INDU_SPECIAL_DB.xlsx')
IT_EFAS_INDU_SPECIAL_DB

,MOTOR_PROD_QUAN,MOTOR_DOME_QUAN,MOTOR_EXPORT_QUAN,MOTOR_EXPORT_SALES,SHIP_ORDER_AMOUNT,SHIP_TONNAGE,SHIP_KAJI,SHIP_BACKLOG,SHIP_BOTTOMS,SHIP_BREAKUP,...,HAEWOON_DUBAI,HAEWOON_DOME_TRANS,CONST_DOME_ORDER_QUAN,CONST_REAL_PRICE,CONST_OVERSEA_ORDER_QUAN,CONST_BREAK_STATE,CONST_UNSOLD_URBAN_x,CONST_UNSOLD_RURAL,CONST_UNSOLD_URBAN_y,CONST_HOUSE_PRICE
STD_YM,,,,,,,,,,,,,,,,,,,,,
201001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,71.84,NaN,7230174.0,6500576.0,22076325,NaN,22235.0,NaN,NaN,78.369310
201002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.33,NaN,4863842.0,6149956.0,3414456,NaN,24280.0,NaN,NaN,78.573339
201003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79.05,NaN,6356428.0,7694396.0,2585419,NaN,23066.0,NaN,NaN,78.746871
201004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.65,NaN,6664740.0,7345162.0,4057530,NaN,22961.0,NaN,NaN,78.884420
201005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,73.28,NaN,8844915.0,7739757.0,847244,NaN,24306.0,NaN,NaN,78.903691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.55,7159.0,17506045.0,12619891.0,NaN,12046.0,3790.0,NaN,NaN,104.459050
202209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.1,6600.0,20834366.0,12857573.0,NaN,11642.0,6272.0,NaN,NaN,103.949855
202210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.04,NaN,9257819.0,13511425.0,NaN,12145.0,NaN,NaN,NaN,103.148521
